In [ ]:
import re
import warnings
from pathlib import Path

import pandas as pd
import requests

dataset_path = Path().resolve() / "datasets"
print(f"Dataset path: {dataset_path}")

data_path = dataset_path / "comstock"
print(f"Data path: {data_path}")

espm_path = dataset_path / "espm"
print(f"ESPM path: {espm_path}")

figures_path = Path().resolve() / "figures"
print(f"Figures path: {figures_path}")

# auto reload any changes in the module
%load_ext autoreload

warnings.filterwarnings("ignore", category=FutureWarning)
# ignore SettingWithCopyWarning
pd.options.mode.chained_assignment = None

In [ ]:
def camel_to_snake(name):
    # Convert lowerCamelCase to snake_case
    s1 = re.sub("(.)([A-Z][a-z]+)", r"\1_\2", name)
    s2 = re.sub("([a-z0-9])([A-Z])", r"\1_\2", s1)
    return s2.lower()


def query_energystar_graphql(graphql_query):
    """Query the Energy Star GraphQL API"""
    url = "https://portfoliomanager.energystar.gov/dataExplorer/graphql"

    headers = {"Content-Type": "application/json", "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36"}

    try:
        response = requests.post(url, headers=headers, json=graphql_query, timeout=30)
        response.raise_for_status()
        return response.json()
    except Exception as e:
        print(f"GraphQL query failed: {e}")
        return None


def build_energystar_query(
    metric="siteEui",
    year_reported=2023,
    group_by="ptSubcategory",
    then_by=None,
    state_province_names=None,
    counties=None,
    year_built_groups=None,
    gfa_groups=None,
    gfa_group2s=None,
    worker_density_groups=None,
    weekly_hours_group=None,
    climate_zone=None,
    csa_city=None,
    csa_area=None,
    pt_categories=None,
    pt_subcategories=None,
    is_energy_star_certified=None,
    with_mean=True,
    with_median=True,
    with_fifth_percentile=True,
    with_twenty_fifth_percentile=True,
    with_seventy_fifth_percentile=True,
    with_ninety_fifth_percentile=True,
):
    """
    Build a complete Energy Star GraphQL query with variables

    Args:
        metric: The metric to query (siteEui, sourceEui, energyStarScore, etc.)
        year_reported: The year to query data for
        group_by: How to group the results (ptSubcategory, climateZone, gfaGroup, etc.)
        then_by: Optional secondary grouping
        year_built_groups: List of year built ranges like ["2000-2009", "1946-1959"]
        weekly_hours_group: List of operating hours ranges like ["48.01 - 60", "40-48"]
        climate_zone: List of climate zones to filter by
        pt_subcategories: List of property subcategories to filter by
        with_*: Boolean flags for which statistics to include in results

    Returns:
        dict: Complete GraphQL query object ready for API call
    """

    # The GraphQL query string
    query = """query GetResults($metric: Metric!, $yearReported: Int!, $groupBy: Grouping!, $thenBy: Grouping, $stateProvinceNames: [String], $counties: [String], $yearBuiltGroups: [String], $gfaGroups: [String], $gfaGroup2s: [String], $csa_city: [String], $csa_area: [String], $climateZone: [String], $isEnergyStarCertified: Boolean, $workerDensityGroups: [String], $weeklyHoursGroup: [String], $ptCategories: [String], $ptSubcategories: [String], $withMean: Boolean!, $withMedian: Boolean!, $withFifthPercentile: Boolean!, $withTwentyFifthPercentile: Boolean!, $withSeventyFifthPercentile: Boolean!, $withNinetyFifthPercentile: Boolean!) {
  getResults(
    metric: $metric
    yearReported: $yearReported
    groupBy: $groupBy
    thenBy: $thenBy
    stateProvinceNames: $stateProvinceNames
    counties: $counties
    yearBuiltGroups: $yearBuiltGroups
    gfaGroups: $gfaGroups
    gfaGroup2s: $gfaGroup2s
    csa_city: $csa_city
    csa_area: $csa_area
    climateZone: $climateZone
    isEnergyStarCertified: $isEnergyStarCertified
    workerDensityGroups: $workerDensityGroups
    weeklyHoursGroup: $weeklyHoursGroup
    ptCategories: $ptCategories
    ptSubcategories: $ptSubcategories
    withMean: $withMean
    withMedian: $withMedian
    withFifthPercentile: $withFifthPercentile
    withTwentyFifthPercentile: $withTwentyFifthPercentile
    withSeventyFifthPercentile: $withSeventyFifthPercentile
    withNinetyFifthPercentile: $withNinetyFifthPercentile
  ) {
    results {
      group
      rowCount
      median @include(if: $withMedian)
      mean @include(if: $withMean)
      fifthPercentile @include(if: $withFifthPercentile)
      twentyFifthPercentile @include(if: $withTwentyFifthPercentile)
      seventyFifthPercentile @include(if: $withSeventyFifthPercentile)
      ninetyFifthPercentile @include(if: $withNinetyFifthPercentile)
      __typename
    }
    count
    __typename
  }
}"""

    # Build the variables object
    variables = {
        "metric": metric,
        "yearReported": year_reported,
        "groupBy": group_by,
        "stateProvinceNames": state_province_names or [],
        "counties": counties or [],
        "yearBuiltGroups": year_built_groups or [],
        "gfaGroups": gfa_groups or [],
        "gfaGroup2s": gfa_group2s or [],
        "workerDensityGroups": worker_density_groups or [],
        "weeklyHoursGroup": weekly_hours_group or [],
        "climateZone": climate_zone or [],
        "csa_city": csa_city or [],
        "csa_area": csa_area or [],
        "ptSubcategories": pt_subcategories or [],
        "ptCategories": pt_categories or [],
        "withMean": with_mean,
        "withMedian": with_median,
        "withFifthPercentile": with_fifth_percentile,
        "withTwentyFifthPercentile": with_twenty_fifth_percentile,
        "withSeventyFifthPercentile": with_seventy_fifth_percentile,
        "withNinetyFifthPercentile": with_ninety_fifth_percentile,
    }

    # Add optional parameters only if they're provided
    if then_by is not None:
        variables["thenBy"] = then_by
    if is_energy_star_certified is not None:
        variables["isEnergyStarCertified"] = is_energy_star_certified

    # Return the complete query object
    return {"operationName": "GetResults", "variables": variables, "query": query}

In [ ]:
# Test the query builder
print("Testing GraphQL query builder...")

group_by = "ptSubcategory"
# group_by = "ptCategory"
metric = "siteEui"
year_reported = 2023
year_built_groups = ["All", "2000-2009", "1946-1959", "1960-1979", "1980-1999", "2010 and after", "Before 1946"]
weekly_hours_group = ["All", "Fewer than 40", "48.01 - 60", "60.01 - 84", "84.01-167", "40-48", "Open Continuously"]
gfa_groups = [
    "All",
    "1,000 - 4,999",
    "5,000 - 9,999",
    "10,000 - 24,999",
    "25,000 - 49,999",
    "50,000 - 99,999",
    "100,000 - 199,999",
    "200,000 - 499,999",
    "500,000 - 999,999",
    "1,000,000+",
]


# store all data in here
all_energystar_data = None

for year_built in year_built_groups:
    for weekly_hours in weekly_hours_group:
        for gfa in gfa_groups:
            print(f"Building query for ENERGY STAR data for year_built={year_built}, weekly_hours={weekly_hours}, gfa={gfa}")
            # if the groups are "All", we want to include all weekly_hours in the query
            if weekly_hours == "All":
                # just take the whole group and drop the "All"
                weekly_hours_q = weekly_hours_group[1:]
            else:
                weekly_hours_q = [weekly_hours]

            if year_built == "All":
                year_built_q = year_built_groups[1:]
            else:
                year_built_q = [year_built]

            if gfa == "All":
                gfa_q = gfa_groups[1:]
            else:
                gfa_q = [gfa]

            query = build_energystar_query(
                then_by="climateZone",
                year_reported=year_reported,
                group_by=group_by,
                metric=metric,
                year_built_groups=year_built_q,
                weekly_hours_group=weekly_hours_q,
                gfa_groups=gfa_q,
            )
            estar_data = query_energystar_graphql(query)
            df_estar = pd.json_normalize(estar_data["data"]["getResults"]["results"])
            # add in the filters to the dataframe
            df_estar["year_reported"] = year_reported
            df_estar["year_built"] = year_built
            df_estar["weekly_hours"] = weekly_hours
            df_estar["gfa"] = gfa

            all_energystar_data = pd.concat([all_energystar_data, df_estar], ignore_index=True)

display(all_energystar_data)

In [ ]:
all_energystar_data_clean = all_energystar_data.copy()

# group,rowCount,median,mean,fifthPercentile,twentyFifthPercentile,seventyFifthPercentile,ninetyFifthPercentile,__typename,year_built,weekly_hours

# remove __typename
all_energystar_data_clean = all_energystar_data_clean.drop(columns=["__typename"])

# split the column `group` into separate columns, apply to every column
new_columns = all_energystar_data_clean["group"].str.split(r"\s*\+\s*", expand=True)
# split the 2nd column again to the first space, which is the climate zone
new_columns = pd.concat([new_columns, new_columns[1].str.split(" ", n=1, expand=True)], axis=1)
new_columns.columns = ["building_subtype", "full_climate_zone", "climate_zone", "climate_zone_description"]
new_columns = new_columns.drop(columns=["full_climate_zone"])
# if building_subtype is All, set climate_zone and climate_zone_description to All
new_columns.loc[new_columns["building_subtype"] == "All", ["climate_zone", "climate_zone_description"]] = "All"
# if climate zone is all, then also set climate_zone_description to All
new_columns.loc[new_columns["climate_zone"] == "All", "climate_zone_description"] = "All"
# put those columns into the DataFrame
all_energystar_data_clean = pd.concat([all_energystar_data_clean, new_columns], axis=1)
# Apply to all column names
all_energystar_data_clean.columns = [camel_to_snake(col) for col in all_energystar_data_clean.columns]


# move the columns into a specific order, drop the original group
all_energystar_data_clean = all_energystar_data_clean.drop(columns=["group"])
reorder_to_middle = ["fifth_percentile", "twenty_fifth_percentile", "median", "mean", "seventy_fifth_percentile", "ninety_fifth_percentile"]
# move these to the beginning first, then they will be in the middle
all_energystar_data_clean = all_energystar_data_clean[
    reorder_to_middle + [col for col in all_energystar_data_clean.columns if col not in reorder_to_middle]
]

move_to_beginning = ["building_subtype", "climate_zone", "year_built", "weekly_hours", "gfa"]
all_energystar_data_clean = all_energystar_data_clean[
    move_to_beginning + [col for col in all_energystar_data_clean.columns if col not in move_to_beginning]
]


reorder_to_end = ["climate_zone_description", "year_reported", "row_count"]
all_energystar_data_clean = all_energystar_data_clean[
    [col for col in all_energystar_data_clean.columns if col not in reorder_to_middle + reorder_to_end] + reorder_to_middle + reorder_to_end
]

# last item, any value of -1 in the `reorder_to_middle` columns needs to be None, not -1
all_energystar_data_clean[reorder_to_middle] = all_energystar_data_clean[reorder_to_middle].replace(-1, None)
# replace 0-5 with 0-5
all_energystar_data_clean["row_count"] = all_energystar_data_clean["row_count"].replace("0-5", "0-5")

# save the data to a CSV file
all_energystar_data_clean.to_csv(espm_path / "energystar_data.csv", index=False)
display(all_energystar_data_clean)